### Control Digital en Sistemas Embebidos - MSE - PRACTICA 4

In [ ]:
import numpy as np
import control as cnt
from scipy.linalg import expm
import matplotlib.pyplot as plt

Dada la siguiente función transferencia de una planta continua:

$H_{s} = (0.2s + 1) / (0.1s^2 + 0.4s + 1)$

**1)** Discretizar la planta.

In [ ]:
r_1 = (9.2e3)
c_1 = 1e-6
r_2 = (18.44e3) #valor particular de la planta
c_2 = 1e-6

h = 0.005
A = [[-(1/r_1+1/r_2)/c_1, 1/(r_2*c_1)],[1/(r_2*c_2), -1/(r_2*c_2)]]
B = [[1/(r_1*c_1)],[0]]
C = [0, 1]

print(A)
print(B)
print(C)

sys = cnt.ss(A,B,C,0)

sysd = cnt.c2d(sys,h)
sys

In [ ]:
hz = cnt.c2d(sys,h)
hz
num, den = cnt.tfdata(hz)
    
DEN0 = den[0][0][0]
DEN1 = den[0][0][1]
DEN2 = den[0][0][2]
NUM0 = num[0][0][0]
NUM1 = num[0][0][1]

#Para uso en real_world.c
print("// Coeficientes del numerador en Q15")
print(f"#define NUM0 Q15_SCALE({NUM0})")
print(f"#define NUM1 Q15_SCALE({NUM1})")
print("\n// Coeficientes del denominador en Q15")
print(f"#define DEN0 Q15_SCALE({DEN0})")
print(f"#define DEN1 Q15_SCALE({DEN1})")
print(f"#define DEN2 Q15_SCALE({DEN2})")

ss = cnt.tf2ss(num, den)
ss

**2)** Obtener la respuesta al escalón en tiempo continuo y discreto.

In [ ]:
t = np.linspace(h, h * 100, 100)
u = np.concatenate((np.zeros(int(len(t) / 2)), np.ones(int(len(t) / 2))), axis=None)

t_hz_open, y_hz_open = cnt.forced_response(hz, t, u)


In [ ]:
p_1 = plt.plot(t_hz_open,y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend('y(t)');

**3)** Aplicar un control diseñado por el método de Pole Placement de forma que el sistema ubique ambos polos en un 30% mas rápido.

Primero probamos sin obtener K0

In [ ]:
Ps = cnt.poles(sys)

# poles = np.exp(Ps*1.3*h)
poles = [0.6 + 0.2j, 0.6 - 0.2j]

K = cnt.place(ss.A, ss.B, poles)
print("k:",K)

Phi_LC = ss.A - ss.B * K

Po = np.array([-10*(np.abs(poles).max()), -11*np.max(np.abs(poles).max())])

Pod = np.exp(Po*h)

L = (cnt.place(ss.A.T, ss.C.T,Pod)).T

print("L:",L)

In [ ]:
hz_pp = cnt.ss2tf(Phi_LC, ss.B, ss.C, ss.D, h)
hz_pp

In [ ]:
t = np.linspace(h, h * 100, 100)
r = np.concatenate((np.zeros(int(len(t) / 2)), np.ones(int(len(t) / 2))), axis=None)

In [ ]:
t_hz_pp, y_hz_pp = cnt.forced_response(hz_pp, t, r)

In [ ]:
p_1 = plt.plot(r)
p_2 = plt.plot(y_hz_pp)
p_3 = plt.plot(y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend((p_1[0], p_2[0], p_3[0]), ('Señal de entrada', 'Salida controlada', 'Respuesta natural del sistema'));

Ahora probamos obteneniendo K0

In [ ]:
K = cnt.place(ss.A, ss.B, poles)

Phi_LC = ss.A - ss.B * K


In [ ]:
Phi_LC = ss.A - ss.B * K
temp1 = np.linalg.inv(np.eye(2) - Phi_LC)
temp2 = np.matmul(ss.C, temp1)
g = np.matmul(temp2, ss.B)
K0 = 1 / g
print("K0: ",K0[0][0])

Gamma_LC = ss.B * K0

In [ ]:
hz_pp = cnt.ss2tf(Phi_LC, Gamma_LC, ss.C, ss.D, h)
hz_pp

**4)** Comparar la respuesta al escalón del punto 2 con la respuesta obtenida aplicando el control por Pole Placement anterior.

In [ ]:
t_hz_pp, y_hz_pp = cnt.forced_response(hz_pp, t, r)

x1 = t_hz_open[y_hz_open> (0.1 * y_hz_open.max())][0]
x2 = t_hz_open[y_hz_open> (0.9 * y_hz_open.max())][0]
y1 = y_hz_open[y_hz_open> (0.1 * y_hz_open.max())][0]
y2 = y_hz_open[y_hz_open> (0.9 * y_hz_open.max())][0]

t_raise_open = x2 - x1
print("Tiempo de subida sin PP",t_raise_open) # Tiempo de subida

x1 = t_hz_pp[y_hz_pp> (0.1 * y_hz_pp.max())][0]
x2 = t_hz_pp[y_hz_pp> (0.9 * y_hz_pp.max())][0]
y1 = y_hz_pp[y_hz_pp> (0.1 * y_hz_pp.max())][0]
y2 = y_hz_pp[y_hz_pp> (0.9 * y_hz_pp.max())][0]
t_raise_pp = x2 - x1
print("Tiempo de subida con PP",t_raise_pp) # Tiempo de subida

print("rel= ",t_raise_open/t_raise_pp)

In [ ]:
p_1 = plt.plot(r)
p_2 = plt.plot(y_hz_pp)
p_3 = plt.plot(y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend((p_1[0], p_2[0], p_3[0]), ('Señal de entrada', 'Salida controlada', 'Respuesta natural del sistema'));

In [ ]:
print("K: ",K[0])
print("K0: ",K0[0][0])
print("L: ",L[:,0])